In [ ]:
# Example of simple model push to Acumos/CMLP

# ===============LICENSE_START=======================================================
# Apache-2.0
# ===================================================================================
# Copyright (C) 2019 AT&T Intellectual Property  All rights reserved.
# ===================================================================================
# This software file is distributed by AT&T
# under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# This file is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ===============LICENSE_END=========================================================


In [ ]:
import numpy as np
import pandas as pd
import os,sys,shutil  # file checks
import dill as pickle   # serialize functions and data as compressed binary 
import gzip  # compression 
import yaml   # configuration file
import time  # time tracking

from acumos.modeling import Model, List, Dict, create_namedtuple, create_dataframe
from acumos.session import AcumosSession, Requirements

# load our configutaion
config_path = 'config.yaml'
if not os.path.isfile(config_path):
    print("Sorry, can't find the configuration file {}, aborting.".format(config_path))
    sys.exit(-1)
config = yaml.safe_load(open(config_path))

# Model Prototype Definition
In this flavor of model wrapping, you can name the function whatever you want and create the input variable pattern that best matches your model's needs.

* **API**: input of a dataframe output of a series of floats

> def classify_review(df: ReviewDataFrame) -> ReviewPredictions:

* **input**: a data frame of revie whelp, a few text fields (review, summary, description), and the categories

> ReviewDataFrame = create_namedtuple('ReviewDataFrame', 
    [('helpful', List[int]), ("reviewText", str), ("summary", str),
     ("unixReviewTime", int), ("categories", List[str]), ("description", str)])

* **output**: a series of floating point values associated with learned model classes/targets

> ReviewPredictions = create_namedtuple(
    'ReviewPredictions', [("prob{}".format(n), float) for n in models["best"].classes_])


## Dynamically generating the model prototype
When you save a model to disk (or push it to the cloud), you are saving a prototype for use.  In fact, one of the steps we'll use in the next notebook is to create a prototype, save it, and use it to call a remote resource.  The reason this is possible is because different `model runners` simple expose the functions that you've created through the prototype.  

Future versions of the library (within Acumos and CMLP alike) include stronger libraries for programatic access with the model primatives more generically. 


In [ ]:
## PART 1 - load models and dump to disk 
# https://pypi.org/project/acumos/#using-dataframes-with-scikit-learn

# load our models
with gzip.open(config["path"]["model_preproc"], 'rb') as f:
    etl = pickle.load(f)
with gzip.open(config["path"]["model_classifier"], 'rb') as f:
    models = pickle.load(f)

# here, an appropriate NamedTuple type is inferred from a pandas DataFrame
print(list(etl["fn_input"].columns))
print(list(etl["fn_input"].dtypes))

ReviewDataFrame = create_namedtuple('ReviewDataFrame', 
    [('helpful', List[int]), ("reviewText", str), ("summary", str),
     ("unixReviewTime", int), ("categories", List[str]), ("description", str)])
# ClassifyDataFrame = create_dataframe('ReviewDataFrame', etl["fn_input"])
# print(ReviewDataFrame)
ReviewPredictions = create_namedtuple(
    'ReviewPredictions', [("probs", List[float]), ("classes", List[str])])
models["classes"] = ["{}".format(c) for c in models["best"].classes_]

def classify_review(df: ReviewDataFrame) -> ReviewPredictions:
    '''Returns an array of review classifications'''
    X = pd.DataFrame([df], columns=ReviewDataFrame._fields)  # we need this because of custom types
    # X = np.column_stack(df)
    X_etl = etl["fn_preproc"](etl, X)  # do classification, don't modify our model
    X_res = models["best"].predict_proba(X_etl)[0]
    # print(models["best"].classes_)
    # print(X_res)
    return ReviewPredictions(X_res, models["classes"])

model = Model(classify=classify_review)



# Publishing and sharing
Congratulations, you're ready to publish your model.  The most simple way to do this is to dump your model to an encapsulated or `wrapped` model file.  We can do this with a single call to the `dump` function.

In [ ]:
# create model so that we can run it locally
session = AcumosSession( config["publish"]["url_model"])
path_dump = os.path.join('data', config["publish"]["name_model2"])
if os.path.exists(path_dump):
    shutil.rmtree(path_dump)
session.dump(model, config["publish"]["name_model2"], 'data')  # creates ~/<name_publish>

## Marketplace
The **marketplace** is a way to share, discover, and facililiate reuse of models created by peers, vendors, etc.  Getting your model to the marketplace starts here, where with a few simple credentials you can wrap and push a model to a known sharing/federation service.

## Logging In
For the main workshop time, we've prepared a sandbox/test server to utilize.  

1. First, click on one of these URLS (test: [http://acumosr-test.research.att.com](http://acumosr-test.research.att.com) or test2: [http://acumosr-test2.research.att.com](http://acumosr-test2.research.att.com)).  In normal operations, you could utilize a different Acumos instance like [http://acumos.research.att.com](http://acumos.research.att.com).

2. Log into Acumos, click on "sign in" an then your user name (top right)

3. Next, click on "Account Settings"

4. Copy the `API Token` and your attuid in this format: `<ATTUID>:<TOKEN>`

5. Update the [config.yaml](config.yaml) settings for `api_token`

In [ ]:
## PART 2 - publish models
os.environ['ACUMOS_TOKEN'] = config["publish"]["api_token"] # visible in this process + all children

# just grab the local scripts for requirements, look at the URL below for more examples
#    of complicated requirements  - https://pypi.org/project/acumos/#declaring-requirements
reqs = Requirements()

# using the AcumosSession created earlier:
time_start = time.time()
session.push(model, config["publish"]["name_model2"], reqs)
print("Model push time... {:0.3f} sec".format(time.time()-time_start))
